Global setup



In [1]:
n=1
k=4
q=41 #Alkaline D
#q should be divisible by 8(?) for NTT
eta=1 #secret key range, prefered 2 in Alkaline
gamma1=2^4 #y coeff range, power of 2, exponent about 2/3 of q?  Earlier version was *not* power of 2

In [2]:
tau = 1 #number of 1's in c
ln(binomial(4, tau)).n()+tau

2.38629436111989

In [3]:
beta = tau*eta
beta

1

Chance of $\mathbf{y}_i$-coefficient passing:

In [4]:
((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k).n()

0.765852644390328

Expected number of repetitions:

In [5]:
1/(((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k))^2.n()

1.70494183535004

Alice generates a signing key

In [6]:
A = random_matrix(ZZ, 4, 4, x=0, y=q)
list(A)

[(12, 16, 3, 10), (27, 24, 25, 20), (15, 16, 20, 4), (36, 10, 9, 29)]

In [8]:
A = matrix(4,4,[(12, 16, 3, 10), (27, 24, 25, 20), (15, 16, 20, 4), (36, 10, 9, 29)])
A

[12 16  3 10]
[27 24 25 20]
[15 16 20  4]
[36 10  9 29]

In [9]:
latex(A)

\left(\begin{array}{rrrr}
12 & 16 & 3 & 10 \\
27 & 24 & 25 & 20 \\
15 & 16 & 20 & 4 \\
36 & 10 & 9 & 29
\end{array}\right)

In [10]:
S1 = random_matrix(ZZ, 4, 4, x=-eta, y=eta+1)
list(S1)

[(-1, -1, -1, 1), (1, 1, 1, 0), (0, -1, 1, -1), (1, 1, 0, -1)]

In [11]:
S1 = matrix(4,4, [(-1, -1, -1, 1), (1, 1, 1, 0), (0, -1, 1, -1), (1, 1, 0, -1)])
S1

[-1 -1 -1  1]
[ 1  1  1  0]
[ 0 -1  1 -1]
[ 1  1  0 -1]

In [12]:
latex(S1)

\left(\begin{array}{rrrr}
-1 & -1 & -1 & 1 \\
1 & 1 & 1 & 0 \\
0 & -1 & 1 & -1 \\
1 & 1 & 0 & -1
\end{array}\right)

In [13]:
S2 = random_matrix(ZZ, 4, 4, x=-eta, y=eta+1)
list(S2)

[(-1, 0, -1, 1), (1, 0, 0, 0), (-1, -1, -1, 1), (0, -1, 0, 1)]

In [14]:
S2= matrix(4,4, [(-1, 0, -1, 1), (1, 0, 0, 0), (-1, -1, -1, 1), (0, -1, 0, 1)])
S2

[-1  0 -1  1]
[ 1  0  0  0]
[-1 -1 -1  1]
[ 0 -1  0  1]

In [16]:
latex(S2)

\left(\begin{array}{rrrr}
-1 & 0 & -1 & 1 \\
1 & 0 & 0 & 0 \\
-1 & -1 & -1 & 1 \\
0 & -1 & 0 & 1
\end{array}\right)

$(S_1, S_2)$ is the private key


In [17]:
T = matrix(((A*S1 + S2) % q))
T

[13 11  6  0]
[18 33 22 23]
[ 4 25 20 33]
[ 3 34 24 40]

In [18]:
latex(T)

\left(\begin{array}{rrrr}
13 & 11 & 6 & 0 \\
18 & 33 & 22 & 23 \\
4 & 25 & 20 & 33 \\
3 & 34 & 24 & 40
\end{array}\right)

$(A,T)$ is the public key


Alice signs the message



In [19]:
M = 'LIFE'
Mlist = [Mod(ord(ch) - ord("A") + 1,q).lift() for ch in M]
Mlist

[12, 9, 6, 5]

In [20]:
y1 = random_matrix(ZZ, 4, 1, x=-(gamma1-1), y=gamma1)
list(y1)

[(14), (-12), (7), (-12)]

In [21]:
y1 = matrix(4,1, [(14), (-12), (7), (-12)])
y1

[ 14]
[-12]
[  7]
[-12]

In [22]:
y2 = random_matrix(ZZ, 4, 1, x=-(gamma1-1), y=gamma1)
list(y2)

[(0), (-1), (13), (6)]

In [23]:
y2 = matrix(4,1, [(0), (-1), (13), (6)])
y2

[ 0]
[-1]
[13]
[ 6]

In [24]:
w1 = matrix(((A*y1+y2) % q)).apply_map(lambda r: Mod(r,q).lift_centered())
w1

[  0]
[-17]
[  0]
[-18]

Start the "non-cryptographic hash/XOF (combined?)"

In [25]:
Mlist

[12, 9, 6, 5]

In [26]:
w1list = [c for row in w1 for c in row]  #flatten the matrix down to coefficients
w1list

[0, -17, 0, -18]

In [27]:
def Dboxraw(list1, list2, itemlen):
    one = vector([1]*len(list1))
    newitem = (2*vector(list1)+one).dot_product(2*vector(list2)+one)  # make sure all bits of input are used
    out = (newitem//2) % (2**itemlen)
    return out

In [28]:
def Dbox(list1, list2, itemlen):
    one = vector([1]*len(list1))
    newitem = (2*vector(list1)+one).dot_product(2*vector(list2)+one)  # make sure all bits of input are used
    out = (newitem//2) % (2**itemlen)
    out = format(out, f'0{itemlen}b')
    return out

In [29]:
Dboxraw(Mlist, w1list, 6)

25

In [30]:
c_hash = Dbox(Mlist, w1list, 6)
c_hash

'011001'

Use Dbox to sample a sparse vector

In [31]:
clenbits = log(k,2)
clenbits

2

In [32]:
def sample_in_ball(digest):  # digest should be at least d bits
    n = 2**clenbits  # variable name conflicts!  This n is local
    c = [0] * n
    k = tau # This k is local
    for i in range(n-tau, n):
        while int(digest[clenbits*k:clenbits*(k+1)], 2) > i:
            k = k+1
        j = int(digest[clenbits*k:clenbits*(k+1)], 2)
        c[i] = c[j]
        c[j] = (-1)^int(digest[i+tau-n], 2)
        k = k+1
    return c

In [33]:
c = matrix(k,1,sample_in_ball(c_hash))
c

[0]
[0]
[1]
[0]

In [34]:
z1 = matrix(y1 + S1*c); z1

[ 13]
[-11]
[  8]
[-12]

Is $\|\mathbf{z_1}\|_\infty \geq \gamma_1-\beta$?

In [35]:
gamma1-beta

15

If so, reject and pick a new $\mathbf{y_1}$

In [36]:
z2 = matrix(y2 + S2*c); z2

[-1]
[-1]
[12]
[ 6]

Is $\|\mathbf{z_2}\|_\infty \geq \gamma_1-\beta$?

In [37]:
gamma1-beta

15

If so, reject and pick a new $\mathbf{y_2}$

If not, $(\mathbf{z_1}, \mathbf{z_2}, \mathbf{c})$ is the signature


In [38]:
(z1, z2, c)

(
[ 13]  [-1]  [0]
[-11]  [-1]  [0]
[  8]  [12]  [1]
[-12], [ 6], [0]
)

In [41]:
latex(z1.transpose())

\left(\begin{array}{rrrr}
13 & -11 & 8 & -12
\end{array}\right)

In [42]:
latex(z2.transpose())

\left(\begin{array}{rrrr}
-1 & -1 & 12 & 6
\end{array}\right)

In [43]:
latex(c.transpose())

\left(\begin{array}{rrrr}
0 & 0 & 1 & 0
\end{array}\right)

Bob verifies the signature

Is $\|\mathbf{z_i}\|_\infty < \gamma_1-\beta$?

In [44]:
z1

[ 13]
[-11]
[  8]
[-12]

In [45]:
z2

[-1]
[-1]
[12]
[ 6]

In [46]:
gamma1-beta

15

Yes

Is $c = H(M \| w_1')$?

In [47]:
w1prime = matrix(((A*z1+z2-T*c) % q)).apply_map(lambda r: Mod(r,q).lift_centered())
w1prime

[  0]
[-17]
[  0]
[-18]

In [48]:
w1primelist = [c for row in w1 for c in row]  #flatten the matrix down to coefficients
w1primelist

[0, -17, 0, -18]

In [49]:
cprime_hash = Dbox(Mlist, w1primelist, 6)
cprime_hash

'011001'

In [50]:
cprime = matrix(k,1,sample_in_ball(cprime_hash))
cprime

[0]
[0]
[1]
[0]

In [51]:
c == cprime

True

Yes.  The signature is accepted.

### Signature which fails:

Is $\|\mathbf{z_i}\|_\infty < \gamma_1-\beta$?

In [53]:
z1 = matrix(4, 1, [5, -3, 13, 10])
z1

[ 5]
[-3]
[13]
[10]

In [55]:
z2 = matrix(4, 1, [-11, -2, 3, 1])
z2

[-11]
[ -2]
[  3]
[  1]

In [46]:
gamma1-beta

15

Yes

Is $c = H(M \| w_1')$?

In [57]:
c = matrix(4, 1, [-1, 0, 0, 0])
c

[-1]
[ 0]
[ 0]
[ 0]

In [58]:
w1prime = matrix(((A*z1+z2-T*c) % q)).apply_map(lambda r: Mod(r,q).lift_centered())
w1prime

[-11]
[-11]
[  6]
[-13]

In [59]:
w1primelist = [c for row in w1 for c in row]  #flatten the matrix down to coefficients
w1primelist

[0, -17, 0, -18]

In [60]:
cprime_hash = Dbox(Mlist, w1primelist, 6)
cprime_hash

'011001'

In [61]:
cprime = matrix(k,1,sample_in_ball(cprime_hash))
cprime

[0]
[0]
[1]
[0]

In [62]:
c == cprime

False

In [63]:
latex(z1.transpose())

\left(\begin{array}{rrrr}
5 & -3 & 13 & 10
\end{array}\right)

In [64]:
latex(z2.transpose())

\left(\begin{array}{rrrr}
-11 & -2 & 3 & 1
\end{array}\right)

In [65]:
latex(c.transpose())

\left(\begin{array}{rrrr}
-1 & 0 & 0 & 0
\end{array}\right)